In [ ]:
import os
import re
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# 이미지 디렉토리 경로 설정
input_dir = "C:/Users/wlstj/Desktop/새 폴더/2. 아이펠(aiffel)/2.개발폴더/images"

# 파일명에서 숫자만 제거하고 특수문자는 유지
def clean_label(label):
    return re.sub(r'\d+', '', label)

# 이미지 파일 경로 수집
input_img_paths = sorted(
    [os.path.join(input_dir, fname)
     for fname in os.listdir(input_dir)
     if fname.endswith(".jpg")]
)

# 클래스 이름 추출 (파일 이름에서 클래스 추출, 숫자만 제거)
class_names = sorted(set(clean_label(fname.split('_')[0]) for fname in os.listdir(input_dir) if fname.endswith('.jpg')))
class_names_dict = {name: idx for idx, name in enumerate(class_names)}

# 이미지와 레이블 로드
images = []
labels = []

for img_path in input_img_paths:
    class_name = clean_label(os.path.basename(img_path).split('_')[0])
    class_id = class_names_dict[class_name]
    image = load_img(img_path, target_size=(224, 224))  # VGG16 모델의 입력 크기
    image = img_to_array(image)
    images.append(image)
    labels.append(class_id)

# Numpy 배열로 변환
images = np.array(images)
labels = np.array(labels)

# 데이터 정규화
images = tf.keras.applications.vgg16.preprocess_input(images)

# 데이터셋 나누기
X_temp, X_test, y_temp, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

# 모델 정의
def create_model(num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # 모든 레이어 동결
    for layer in base_model.layers:
        layer.trainable = False

    # 마지막 몇 레이어의 동결 해제
    # 동결 해제할 레이어의 수를 조정할 수 있습니다.
    for layer in base_model.layers[-4:]:  # 마지막 30개 레이어의 동결 해제 예시
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# 클래스 수 정의
num_classes = len(class_names)

# 모델 생성
model = create_model(num_classes)

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# 모델 훈련
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# 모델 저장
model_path = 'vgg16_model.h5'
model.save(model_path)

# 모델 로드 및 예측 함수
def load_and_predict(model_path, img_path):
    try:
        model = tf.keras.models.load_model(model_path, custom_objects={
            'VGG16': VGG16,
            'GlobalAveragePooling2D': GlobalAveragePooling2D,
            'Dense': Dense,
            'BatchNormalization': BatchNormalization,
            'Dropout': Dropout
        })

        img = load_img(img_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.vgg16.preprocess_input(img_array)

        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        predicted_label = class_names[predicted_class]

        return {
            "predicted_label": predicted_label,
            "prediction_score": f"{predictions[0][predicted_class]:.5f}"
        }
    except Exception as e:
        return {"error": str(e)}

# 모델 훈련 후 history 객체에 기록된 손실 및 정확도 시각화
def plot_training_history(history):
    plt.figure(figsize=(12, 5))
    
    # 훈련 손실과 검증 손실
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # 훈련 정확도와 검증 정확도
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# 훈련이 완료된 후 history 객체를 사용하여 시각화
plot_training_history(history)
